In [2]:
%%capture
!pip install rioxarray "xarray[complete]" dask-expr
!pip install dask-expr

In [3]:
# Importing libraries
import xarray as xr
import os
import pandas as pd
import numpy as np
import pandas as pd
from dask import dataframe as ddf
from dask.diagnostics import ProgressBar


In [4]:
sourcePath = 'data/iron/'
destinationPath = 'data/brozen/'

standardizedResolutionParameter =   xr.Dataset(
    {
        "lat": (["lat"], np.arange(-90, 90, 0.05)),
        "lon": (["lon"], np.arange(-180, 180, 0.05)),
    }
)


lat =  np.arange(-90, 90, 0.05)
lon =  np.arange(-180, 180, 0.05)

In [116]:
def getFilesForProcessing(path:str) -> pd.DataFrame:
    _listFiles = os.listdir(path)
    _dfFiles = pd.DataFrame(_listFiles,columns=['files'])
    _dfFiles['path']  = path
    _dfFiles['fullNamePath'] = _dfFiles['path']+_dfFiles['files']
    return _dfFiles.drop(columns=['files','path'])

def interpolateResolution(filePath:str
                           , interpolationMethod:str ='linear'
                           , destinationPath:str=None
                          ):
    global standardizedResolutionParameter
    _fileName = filePath.split('/')[-1]
    _data = xr.open_dataset(filePath)
    _data = _data.interp(lat=np.arange(-90, 90, 0.05)
                            ,lon = np.arange(-180, 180, 0.05)
                         ,method=interpolationMethod)
    _data.to_netcdf(destinationPath+_fileName)
    
    
    
    

In [117]:
filesForProcess = getFilesForProcessing('data/originalnc/')

In [8]:
filesForProcess.iloc[0]

fullNamePath    data/originalnc/Solar-Radiation-Flux_C3S-glob-...
Name: 0, dtype: object

In [118]:
filesForProcessDask = ddf.from_pandas(filesForProcess
                                      ,npartitions = 50)

In [119]:
with ProgressBar():
    filesForProcessDask.map_partitions(
        lambda df:df.fullNamePath.apply(lambda f: interpolateResolution(filePath=f
                                                                 ,destinationPath=destinationPath))
        ,meta = { 'name' : object  }
    ).compute(scheduler='processes')

[                                        ] | 0% Completed | 2hr 42mss


Exception ignored in: <function CachingFileManager.__del__ at 0x7fc8c7da8820>
Traceback (most recent call last):
  File "/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/xarray/backends/file_manager.py", line 236, in __del__
    def __del__(self) -> None:
KeyboardInterrupt: 

KeyboardInterrupt



In [12]:
interpolateResolution('data/originalnc/Solar-Radiation-Flux_C3S-glob-agric_AgERA5_19860304_final-v1.0.nc'
                           ,'cubic'
                           , destinationPath
                          )

PermissionError: [Errno 13] Permission denied: '/home/ec2-user/SageMaker/datalake/notebooks/utils/data/brozen/Solar-Radiation-Flux_C3S-glob-agric_AgERA5_19860304_final-v1.0.nc'

In [21]:
"s3://climate-action-datalake/zone=temporal/"

In [7]:
b = xr.open_dataset(destinationPath+'Solar-Radiation-Flux_C3S-glob-agric_AgERA5_20010206_final-v1.0.nc')

In [8]:
b

<xarray.Dataset> Size: 26MB
Dimensions:               (time: 1, lon: 3600, lat: 1801)
Coordinates:
  * time                  (time) datetime64[ns] 8B 2001-02-06
  * lon                   (lon) float64 29kB -180.0 -179.9 ... 179.8 179.9
  * lat                   (lat) float64 14kB 90.0 89.9 89.8 ... -89.9 -90.0
Data variables:
    Solar_Radiation_Flux  (time, lat, lon) float32 26MB ...
Attributes:
    CDI:          Climate Data Interface version 1.9.2 (http://mpimet.mpg.de/...
    history:      Mon Mar 15 19:46:58 2021: cdo splitday /archive/ESG/wit015/...
    Conventions:  CF-1.7
    CDO:          Climate Data Operators version 1.9.2 (http://mpimet.mpg.de/...

In [43]:
s = xr.open_dataset('data/originalnc/Solar-Radiation-Flux_C3S-glob-agric_AgERA5_19860304_final-v1.0.nc')
s = s.dropna( dim = 'lat'
              , how='all')
s = s.dropna(dim ='lon' 
               , how='all')
s = s.interpolate_na(dim = 'lat'
                     ,method ='cubic'
                     ,use_coordinate = False)
s = s.interpolate_na(dim = 'lon'
                     ,use_coordinate = False
                     ,method ='cubic')
s = s.interpolate_na(dim = ['lon','lat'], use_coordinate = False
                     ,method ='cubic')
d = s.interp(lat = np.arange(-90, 90, 0.05)
             ,lon = np.arange(-180, 180, 0.05)
             , kwargs={"fill_value": "extrapolate"}
             ,method='linear')

In [44]:
df = s.to_dataframe()

In [45]:
d

<xarray.Dataset> Size: 207MB
Dimensions:               (time: 1, lat: 3600, lon: 7200)
Coordinates:
  * time                  (time) datetime64[ns] 8B 1986-03-04
  * lat                   (lat) float64 29kB -90.0 -89.95 -89.9 ... 89.9 89.95
  * lon                   (lon) float64 58kB -180.0 -179.9 ... 179.9 180.0
Data variables:
    Solar_Radiation_Flux  (time, lat, lon) float64 207MB 8.796e+06 ... nan
Attributes:
    CDI:          Climate Data Interface version 1.9.2 (http://mpimet.mpg.de/...
    history:      Fri Mar 12 13:49:09 2021: cdo splitday /archive/ESG/wit015/...
    Conventions:  CF-1.7
    CDO:          Climate Data Operators version 1.9.2 (http://mpimet.mpg.de/...

In [57]:
y = d.to_dataframe()
y[y.Solar_Radiation_Flux > 0]

Solar_Radiation_Flux
time       lon    lat                        
1986-03-04 -180.0  71.8            3886064.50
                   71.7            3916857.25
                   71.6            3926533.25
                   71.5            3643289.25
                   71.4            3647514.00
...                                       ...
            179.9 -89.6            8553266.00
                  -89.7            8514890.00
                  -89.8            8475249.00
                  -89.9            8755919.00
                  -90.0            8796078.00

[2353347 rows x 1 columns]

In [45]:
standardizedResolutionParameter

<xarray.Dataset> Size: 86kB
Dimensions:  (lat: 3600, lon: 7200)
Coordinates:
  * lat      (lat) float64 29kB -90.0 -89.95 -89.9 -89.85 ... 89.85 89.9 89.95
  * lon      (lon) float64 58kB -180.0 -179.9 -179.9 ... 179.9 179.9 180.0
Data variables:
    *empty*

In [58]:
xr.open_dataset('19810106.nc')

<xarray.Dataset> Size: 58MB
Dimensions:        (lon: 7200, lat: 2000, time: 1, band: 1)
Coordinates:
  * lon            (lon) float64 58kB -180.0 -179.9 -179.9 ... 179.9 179.9 180.0
  * lat            (lat) float64 16kB 49.97 49.92 49.87 ... -49.88 -49.93 -49.98
  * time           (time) datetime64[ns] 8B 1981-01-06
Dimensions without coordinates: band
Data variables:
    precipitation  (time, band, lat, lon) float32 58MB ...